In [1]:
import cv2
import time

In [2]:
classNames = []
classFile = "coco.names"

with open(classFile, "rt") as f:
    classNames = f.read().rstrip("\n").split("\n")
# print(classNames)

configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
weightsPath = "frozen_inference_graph.pb"

In [3]:
# SETTINGS
# img = cv2.imread("wick.jpg")
# cap = cv2.VideoCapture()
# cap.set(3, 640)
# cap.set(4, 480)

# net setup
net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)


# frame rate setup
last_frame_time = 0
frame_interval = 0.2  # seconds

In [12]:
cap = cv2.VideoCapture(0)
while True:
    try:

        # if the frame wait interval is longer than 1s, the program will sleep for 90% of the wait and update time before moving on.
        current_time = time.time()
        if frame_interval > 1 and current_time - last_frame_time < frame_interval:
            time.sleep(0.9 * frame_interval)
            current_time = time.time()

        # else it will only
        if current_time - last_frame_time >= frame_interval:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # detect objects
            classIds, confs, bbox = net.detect(frame, confThreshold=0.5)

            # add rectagle and label to each object
            try:
                for classId, confidence, box in zip(
                    classIds.flatten(), confs.flatten(), bbox
                ):
                    # print(box)
                    cv2.rectangle(frame, box, color=(0, 255, 0), thickness=2)
                    cv2.putText(
                        frame,
                        classNames[classId - 1],
                        (box[0] + 10, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX,
                        1,
                        (255, 0, 0),
                        2,
                    )
            except:
                # nothing detected. continue to next image.
                continue

            # render image
            cv2.imshow("output", frame)

            last_frame_time = current_time

        # quit
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    except Exception as e:
        cap.release()
        cv2.destroyAllWindows()
        print("Something went wrong: ", e)

# close image window
cap.release()
cv2.destroyAllWindows()

In [14]:
img = cv2.imread("wick.jpg")

for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
    cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
while True:

    cv2.imshow("output", img)
    cv2.putText(
        img,
        classNames[classId - 1],
        (box[0] + 10, box[1] + 30),
        cv2.FONT_HERSHEY_COMPLEX,
        1,
        (255, 0, 0),
        2,
    )
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()

In [23]:
%%timeit

time.sleep(0.01)

10.1 ms ± 7.63 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
